In [2]:
import numpy as np
from cities.utils.cleaning_utils import find_repo_root
import pandas as pd
import numpy as np
import requests

root = find_repo_root()

In [89]:
area = 'metropolitan%20statistical%20area/micropolitan%20statistical%20area:*'

variables = "DP05_0070E,DP05_0072E,DP05_0073E,DP05_0074E,DP05_0075E,DP05_0077E,DP05_0078E,DP05_0079E,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E"


url = f'https://api.census.gov/data/2021/acs/acs5/profile?get={variables}&for={area}'

response = requests.get(url)

assert response.status_code == 200 # 200 means success

data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])
df.columns


Index(['DP05_0070E', 'DP05_0072E', 'DP05_0073E', 'DP05_0074E', 'DP05_0075E',
       'DP05_0077E', 'DP05_0078E', 'DP05_0079E', 'DP05_0080E', 'DP05_0081E',
       'DP05_0082E', 'DP05_0083E',
       'metropolitan statistical area/micropolitan statistical area'],
      dtype='object')

In [90]:
df.columns = [
        "total_pop",
        "mexican",
        "puerto_rican",
        "cuban",
        "other_hispanic_latino",
        "white",
        "black_african_american",
        "american_indian_alaska_native",
        "asian",
        "native_hawaiian_other_pacific_islander",
        "other_race",
        "two_or_more_sum",
        'CBSA',
    ]

df = df[['CBSA', 'total_pop', 'mexican', 'puerto_rican', 'cuban', 'other_hispanic_latino', 'white', 'black_african_american', 'american_indian_alaska_native', 'asian', 'native_hawaiian_other_pacific_islander', 'other_race', 'two_or_more_sum']]
df.head()





,CBSA,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum
0,10100,42476,446,355,169,498,36680,909,1316,1302,61,4,736
1,10140,74737,5844,234,94,1707,58563,947,2646,1020,277,48,3357
2,10180,175241,38061,1110,391,3267,110600,12846,249,3329,58,890,4440
3,10220,38102,1941,12,0,256,24236,936,6203,332,59,43,4084
4,10300,99346,6545,886,68,859,85294,2036,135,276,38,186,3023


In [91]:
df.to_csv(f"{root}/data/raw/ethnic_composition_cbsa.csv", index=False)

In [128]:
df = pd.read_csv(f"{root}/data/raw/ethnic_composition_cbsa.csv")

In [129]:
# further wrangling

metro_areas = pd.read_csv(f"{root}/data/raw/metrolist.csv")
metro_areas.head()

,GeoFIPS,GeoName,CountyFIPS,CountyName
0,10180,"Abilene, TX (MA)",48059,"Callahan, TX"
1,10180,"Abilene, TX (MA)",48253,"Jones, TX"
2,10180,"Abilene, TX (MA)",48441,"Taylor, TX"
3,10420,"Akron, OH (MA)",39133,"Portage, OH"
4,10420,"Akron, OH (MA)",39153,"Summit, OH"


In [130]:
df['CBSA'] = df['CBSA'].astype(np.int64)
df2 = df[df['CBSA'].isin(metro_areas['GeoFIPS'])]



# 

#df2.head()

df2 = pd.merge(df2, metro_areas[['GeoFIPS', 'GeoName']], left_on='CBSA', right_on='GeoFIPS', how='inner')
df2 = df2.drop_duplicates(subset=['CBSA'])

df2.drop(columns='CBSA', inplace=True)
df2.head()




,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum,GeoFIPS,GeoName
0,175241,38061,1110,391,3267,110600,12846,249,3329,58,890,4440,10180,"Abilene, TX (MA)"
3,702464,6617,4470,350,4493,551208,83056,642,23358,70,1824,26376,10420,"Akron, OH (MA)"
5,149298,1882,615,270,1639,61074,78470,171,2167,41,216,2753,10500,"Albany, GA (MA)"
9,127200,10659,254,70,1316,106456,465,733,1253,149,207,5638,10540,"Albany-Lebanon, OR (MA)"
10,898265,5466,22155,2006,19300,705697,64408,915,40654,227,4271,33166,10580,"Albany-Schenectady-Troy, NY (MA)"


In [135]:

cols_to_save = df2.shape[1] - 2
ethnic_composition_ma = df2[['GeoFIPS', 'GeoName'] + list(df2.columns[0:cols_to_save])]
ethnic_composition_ma.head()



,GeoFIPS,GeoName,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum
0,10180,"Abilene, TX (MA)",175241,38061,1110,391,3267,110600,12846,249,3329,58,890,4440
3,10420,"Akron, OH (MA)",702464,6617,4470,350,4493,551208,83056,642,23358,70,1824,26376
5,10500,"Albany, GA (MA)",149298,1882,615,270,1639,61074,78470,171,2167,41,216,2753
9,10540,"Albany-Lebanon, OR (MA)",127200,10659,254,70,1316,106456,465,733,1253,149,207,5638
10,10580,"Albany-Schenectady-Troy, NY (MA)",898265,5466,22155,2006,19300,705697,64408,915,40654,227,4271,33166


In [136]:
ethnic_composition_ma.iloc[:, 2:] = ethnic_composition_ma.iloc[:, 2:].apply(
    pd.to_numeric, errors="coerce"
)
ethnic_composition_ma[ethnic_composition_ma.columns[2:]] = ethnic_composition_ma[
    ethnic_composition_ma.columns[2:]
].astype(float)

ethnic_composition_ma["other_race_races"] = (
    ethnic_composition_ma["other_race"] + ethnic_composition_ma["two_or_more_sum"]
)
ethnic_composition_ma = ethnic_composition_ma.drop(
    ["other_race", "two_or_more_sum"], axis=1
)

ethnic_composition_ma["totalALT"] = ethnic_composition_ma.iloc[:, 3:].sum(axis=1)
assert (ethnic_composition_ma["totalALT"] == ethnic_composition_ma["total_pop"]).all()
ethnic_composition_ma = ethnic_composition_ma.drop("totalALT", axis=1)

row_sums = ethnic_composition_ma.iloc[:, 2:].sum(axis=1)
ethnic_composition_ma.iloc[:, 3:] = ethnic_composition_ma.iloc[:, 3:].div(
    row_sums, axis=0
)

ethnic_composition_ma = ethnic_composition_ma.drop(["total_pop"], axis=1)

In [137]:
ethnic_composition_ma



,GeoFIPS,GeoName,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race_races
0,10180,"Abilene, TX (MA)",0.108596,0.003167,0.001116,0.009321,0.315565,0.036652,0.000710,0.009498,0.000165,0.015208
3,10420,"Akron, OH (MA)",0.004710,0.003182,0.000249,0.003198,0.392339,0.059118,0.000457,0.016626,0.000050,0.020072
5,10500,"Albany, GA (MA)",0.006303,0.002060,0.000904,0.005489,0.204537,0.262797,0.000573,0.007257,0.000137,0.009943
9,10540,"Albany-Lebanon, OR (MA)",0.041899,0.000998,0.000275,0.005173,0.418459,0.001828,0.002881,0.004925,0.000586,0.022976
10,10580,"Albany-Schenectady-Troy, NY (MA)",0.003043,0.012332,0.001117,0.010743,0.392811,0.035851,0.000509,0.022629,0.000126,0.020839
...,...,...,...,...,...,...,...,...,...,...,...,...
1152,49420,"Yakima, WA (MA)",0.240003,0.001043,0.000343,0.011605,0.207681,0.004591,0.013086,0.004542,0.002185,0.014921
1153,49620,"York-Hanover, PA (MA)",0.007849,0.020886,0.000781,0.011631,0.408436,0.026912,0.000272,0.007101,0.000179,0.015953
1154,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",0.003811,0.011575,0.000539,0.003176,0.409910,0.050231,0.000384,0.003791,0.000171,0.016410
1157,49700,"Yuba City, CA (MA)",0.137461,0.003229,0.000368,0.013138,0.239620,0.012399,0.003869,0.059872,0.001922,0.028122
